<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c804c100b9ef96cfeecf06307d0196c1a3337e55c07ef2f8660a704677eef8db
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-08 13:20:20
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: -2.18 L (-1.54%)
Current PnL: -29.15 L (-18.83%)
CY Booked + Current PnL: -14.86 L (-9.6%)
-------------------
Total profit:  1.29 L
Total loss:  -30.43 L
-------------------
Total Booked + Current PnL: 12.12 L (9.48%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.27%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.78 L (67.41%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.12,-16.72,20.18,0.09,16937.0,-16845.0,83932.0,94.88,38.0,M-SC,2.38,253.0,-0.99,0.63,10.27,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-1.44,-13.50,15.63,0.02,23601.0,-23564.0,150995.0,130.47,49.0,M-SC,16.81,234.0,-1.00,1.13,31.34,OX40N,NTT,PAINTS
43,ITC,452.00,-0.65,-1.71,12.35,10.44,24296.0,-3413.0,196725.0,-41.15,45.0,X-LC,1.60,5.0,-0.14,1.47,3.76,X40,NTT,FMCG
50,MASFIN,398.61,-1.92,-3.72,26.76,22.05,25244.0,-3645.0,94335.0,-16.89,52.0,H-SC,6.85,164.0,-0.14,0.70,37.12,XR,ATH,FINANCE
35,ICICIGI,2252.93,-0.83,4.46,15.06,20.19,27423.0,7774.0,182094.0,-18.14,42.0,X-MC,6.15,68.0,0.28,1.36,20.25,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,STARHEALTH,761.00,1.04,-10.48,64.50,47.26,162317.0,-29469.0,251654.0,22.16,35.0,H-SC,13.78,171.0,-0.18,1.88,34.83,XY24,NTT,INSURANCE
32,HCLTECH,1922.01,0.49,9.75,13.65,24.73,36243.0,23594.0,265518.0,14.46,72.0,X-LC,5.55,7.0,0.65,1.98,26.47,X40,ATH,IT
84,WIPRO,420.00,0.42,7.05,60.92,72.27,110506.0,11947.0,181395.0,-8.35,74.0,M-LC,2.80,99.0,0.11,1.35,13.66,XR,NTT,IT
73,TCS,4389.97,0.06,-11.96,35.49,19.28,108092.0,-41387.0,304569.0,-23.30,70.0,X-LC,6.05,1.0,-0.38,2.27,12.59,X40,ATH,IT
54,PGHH,17907.65,0.00,-5.98,42.27,33.76,79808.0,-12015.0,188805.0,-33.72,29.0,X-MC,5.06,57.0,-0.15,1.41,0.00,X40,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,KPIGREEN,730.66,-7.96,-21.59,87.42,46.95,85883.0,-27055.0,98242.0,-27.45,25.0,H-SC,4.29,154.0,-0.32,0.73,23.07,MH,ATH,POWER
0,5PAISA,593.00,-4.13,-42.61,96.72,12.90,121290.0,-93093.0,125403.0,85.10,39.0,H-SC,11.52,161.0,-0.77,0.93,4.27,OX40N,NTT,FINANCE
18,COFFEEDAY,80.00,-3.81,-45.77,149.45,35.27,92022.0,-51975.0,61574.0,-57.75,21.0,L-SC,12.92,270.0,-0.56,0.46,50.70,XR,NTT,HOTELS
57,RAJOOENG,143.10,-3.79,-34.54,113.26,39.61,75997.0,-35400.0,67100.0,-55.65,26.0,H-SC,25.16,135.0,-0.47,0.50,0.00,AR,ATH,MISC
64,SHALBY,327.00,-3.56,-26.72,70.72,25.10,93872.0,-48406.0,132737.0,876.08,20.0,M-SC,12.73,250.0,-0.52,0.99,6.38,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-2.56,-9.10,128.72,107.91,172755.0,-13434.0,134210.0,-26.23,24.0,M-SC,11.62,216.0,-0.08,1.00,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.44,-13.50,15.63,0.02,23601.0,-23564.0,150995.0,130.47,49.0,M-SC,16.81,234.0,-1.00,1.13,31.34,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.12,-16.72,20.18,0.09,16937.0,-16845.0,83932.0,94.88,38.0,M-SC,2.38,253.0,-0.99,0.63,10.27,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.80,-25.24,51.79,13.47,104409.0,-68067.0,201600.0,-69.28,27.0,H-SC,2.87,158.0,-0.65,1.50,2.51,XY24,NTT,PAINTS
66,SIS,528.00,-1.69,-25.70,63.21,21.26,52143.0,-28540.0,82492.0,1944.04,42.0,H-SC,4.26,166.0,-0.55,0.61,11.46,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.42,7.05,60.92,72.27,110506.0,11947.0,181395.0,-8.35,74.0,M-LC,2.8,99.0,0.11,1.35,13.66,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.42,7.05,60.92,72.27,110506.0,11947.0,181395.0,-8.35,74.0,M-LC,2.80,99.0,0.11,1.35,13.66,XR,NTT,IT
38,INDIAMART,4810.62,-2.48,-3.10,113.33,106.72,135446.0,-3821.0,119515.0,-51.96,33.0,H-SC,1.31,139.0,-0.03,0.89,19.56,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.77,-2.54,39.16,35.63,79988.0,-5315.0,204259.0,-16.30,37.0,H-MC,3.18,119.0,-0.07,1.52,13.71,AR,ATH,PHARMA
37,IEX,219.00,-2.52,-3.87,54.61,48.63,104971.0,-7735.0,192219.0,-36.00,49.0,H-SC,14.13,136.0,-0.07,1.43,7.24,XR,NTT,MISC
25,FINCABLES,1641.55,-2.56,-9.10,128.72,107.91,172755.0,-13434.0,134210.0,-26.23,24.0,M-SC,11.62,216.0,-0.08,1.00,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-2.04,-23.44,93.91,48.45,144156.0,-47002.0,153504.0,-28.89,10.0,X-MC,10.65,64.0,-0.33,1.14,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.75,-38.72,120.63,35.20,95117.0,-49820.0,78850.0,3.39,23.0,X-SC,14.97,92.0,-0.52,0.59,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.85,-12.45,38.88,21.58,47701.0,-17451.0,122688.0,-55.87,25.0,X-MC,5.35,56.0,-0.37,0.91,8.62,X40N,ATH,IT
46,JSWINFRA,342.00,-2.51,-11.56,30.29,15.23,53670.0,-23159.0,177188.0,-24.28,26.0,X-MC,7.57,66.0,-0.43,1.32,16.96,X40N,NTT,REALTY
75,TMPV,600.00,-1.27,-17.27,71.87,42.18,164339.0,-47749.0,228661.0,-25.52,27.0,X-LC,4.52,3.0,-0.29,1.70,0.00,XY24,NTT,AUTO


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-0.87,-20.69,93.54,53.50,223863.0,-62428.0,239323.0,-61.59,34.0,X-MC,1.36,58.0,-0.28,1.78,6.30,XY24,NTT,FMCG
43,ITC,452.0,-0.65,-1.71,12.35,10.44,24296.0,-3413.0,196725.0,-41.15,45.0,X-LC,1.60,5.0,-0.14,1.47,3.76,X40,NTT,FMCG
3,ACC,3906.0,-0.07,-24.48,117.36,64.15,210896.0,-58251.0,179700.0,-55.88,38.0,X-SC,1.68,82.0,-0.28,1.34,0.80,XY24,BTT,CEMENT
15,CAMS,4762.0,-3.22,-80.33,534.34,24.79,264744.0,-202310.0,49546.0,-81.32,40.0,X-SC,2.08,86.0,-0.76,0.37,21.12,X40N,NTT,MISC
36,ICICIPRULI,790.0,-0.34,3.43,26.62,30.97,48662.0,6065.0,182803.0,-20.92,56.0,X-MC,2.20,75.0,0.12,1.36,16.42,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.75,-38.72,120.63,35.20,95117.0,-49820.0,78850.0,3.39,23.0,X-SC,14.97,92.0,-0.52,0.59,0.0,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-2.04,-23.44,93.91,48.45,144156.0,-47002.0,153504.0,-28.89,10.0,X-MC,10.65,64.0,-0.33,1.14,0.0,X40N,ATH,FINANCE
55,QUESS,424.00,-0.89,-32.61,111.96,42.84,49049.0,-21197.0,43809.0,-55.92,32.0,X-SC,37.31,83.0,-0.43,0.33,0.0,XY24,NTT,MISC
53,PAGEIND,51605.07,-0.89,-9.32,39.02,26.07,57937.0,-15260.0,148480.0,-33.27,30.0,X-MC,11.19,55.0,-0.26,1.11,0.0,X40,ATH,APPARELS
54,PGHH,17907.65,0.00,-5.98,42.27,33.76,79808.0,-12015.0,188805.0,-33.72,29.0,X-MC,5.06,57.0,-0.15,1.41,0.0,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.89,-32.61,111.96,42.84,49049.0,-21197.0,43809.0,-55.92,32.0,X-SC,37.31,83.0,-0.43,0.33,0.00,XY24,NTT,MISC
15,CAMS,4762.00,-3.22,-80.33,534.34,24.79,264744.0,-202310.0,49546.0,-81.32,40.0,X-SC,2.08,86.0,-0.76,0.37,21.12,X40N,NTT,MISC
59,RELAXO,1176.00,-1.43,-48.36,194.96,52.31,146134.0,-70204.0,74956.0,-45.02,31.0,X-SC,5.76,91.0,-0.48,0.56,0.33,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.75,-38.72,120.63,35.20,95117.0,-49820.0,78850.0,3.39,23.0,X-SC,14.97,92.0,-0.52,0.59,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.85,-12.45,38.88,21.58,47701.0,-17451.0,122688.0,-55.87,25.0,X-MC,5.35,56.0,-0.37,0.91,8.62,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.06,-11.96,35.49,19.28,108092.0,-41387.0,304569.0,-23.30,70.0,X-LC,6.05,1.0,-0.38,2.27,12.59,X40,ATH,IT
41,INFY,2275.00,-0.12,9.89,40.94,54.87,140762.0,30937.0,343825.0,-14.57,70.0,X-LC,3.39,2.0,0.22,2.56,18.99,X40,BTT,IT
75,TMPV,600.00,-1.27,-17.27,71.87,42.18,164339.0,-47749.0,228661.0,-25.52,27.0,X-LC,4.52,3.0,-0.29,1.70,0.00,XY24,NTT,AUTO
81,VBL,671.64,-2.82,-5.79,44.01,35.67,130957.0,-18279.0,297563.0,-17.02,49.0,X-LC,4.94,4.0,-0.14,2.22,7.27,X40N,ATH,FMCG
43,ITC,452.00,-0.65,-1.71,12.35,10.44,24296.0,-3413.0,196725.0,-41.15,45.0,X-LC,1.60,5.0,-0.14,1.47,3.76,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-2.45,-19.24,124.96,81.67,94670.0,-18050.0,75760.0,6805.56,32.0,L-SC,15.49,271.0,-0.19,0.56,46.71,XR,NTT,CERAMICS
50,MASFIN,398.61,-1.92,-3.72,26.76,22.05,25244.0,-3645.0,94335.0,-16.89,52.0,H-SC,6.85,164.0,-0.14,0.70,37.12,XR,ATH,FINANCE
13,BSOFT,831.70,-0.24,-21.73,92.66,50.79,102004.0,-30569.0,110084.0,0.50,72.0,H-SC,5.59,151.0,-0.30,0.82,28.17,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.44,-13.50,15.63,0.02,23601.0,-23564.0,150995.0,130.47,49.0,M-SC,16.81,234.0,-1.00,1.13,31.34,OX40N,NTT,PAINTS
68,STARHEALTH,761.00,1.04,-10.48,64.50,47.26,162317.0,-29469.0,251654.0,22.16,35.0,H-SC,13.78,171.0,-0.18,1.88,34.83,XY24,NTT,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.24,-21.73,92.66,50.79,102004.0,-30569.0,110084.0,0.50,72.0,H-SC,5.59,151.0,-0.30,0.82,28.17,XR,ATH,IT
5,ASIANPAINT,3460.25,-0.85,2.88,17.56,20.95,45484.0,7251.0,259019.0,1.90,74.0,X-LC,18.90,31.0,0.16,1.93,39.54,X40,ATH,PAINTS
32,HCLTECH,1922.01,0.49,9.75,13.65,24.73,36243.0,23594.0,265518.0,14.46,72.0,X-LC,5.55,7.0,0.65,1.98,26.47,X40,ATH,IT
84,WIPRO,420.00,0.42,7.05,60.92,72.27,110506.0,11947.0,181395.0,-8.35,74.0,M-LC,2.80,99.0,0.11,1.35,13.66,XR,NTT,IT
73,TCS,4389.97,0.06,-11.96,35.49,19.28,108092.0,-41387.0,304569.0,-23.30,70.0,X-LC,6.05,1.0,-0.38,2.27,12.59,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.90
1,25,45.87
2,50,76.80


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.87
MC    30.23
LC    24.92
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.33
X40      22.64
X40N     10.82
XR       10.00
XY25      9.38
AR        9.37
OX40N     7.74
SR        1.01
MH        0.73
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.38
X-MC    23.50
X-LC    21.33
M-SC    12.13
X-SC     6.98
H-MC     4.96
M-MC     1.41
L-SC     1.38
M-LC     1.35
H-LC     1.20
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.44,-7.60,42.39
IT,12.94,-16.04,78.78
FINANCE,9.52,-21.22,72.44
ELECTRICAL,6.24,-11.95,52.96
PAINTS,6.19,-11.24,27.76
MISC,6.12,-58.07,122.82
INSURANCE,4.60,-2.54,38.67
PHARMA,4.09,-1.81,34.39
AUTO,3.45,-23.99,77.16


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3236844.0,21
XR,1321232.0,13
AR,1314560.0,10
X40,1002581.0,14
X40N,995870.0,9
OX40N,754011.0,10
XY25,382161.0,6
SR,284902.0,2
MH,85883.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3709059.0,25
M-SC,1433089.0,15
X-MC,1329151.0,16
X-SC,985515.0,8
X-LC,899232.0,11
H-MC,412771.0,3
L-SC,275658.0,3
M-LC,110506.0,1
H-LC,72257.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1298346.0      6
           AR         899819.0      5
M-SC       XY24       812765.0      6
H-SC       XR         793124.0      7
X-SC       X40N       506125.0      3
X-MC       X40        462668.0      7
           XY24       393415.0      3
X-LC       X40        354877.0      5
H-SC       OX40N      346985.0      4
M-SC       OX40N      318060.0      5
X-SC       XY24       294354.0      3
H-SC       SR         284902.0      2
X-MC       X40N       282474.0      4
X-LC       XY24       241579.0      2
H-MC       AR         216386.0      2
X-LC       X40N       207271.0      2
H-MC       XY24       196385.0      1
X-MC       XY25       190594.0      2
L-SC       XR         186692.0      2
X-SC       X40        185036.0      2
M-SC       XR         176166.0      2
           AR         126098.0      2
M-LC       XR         110506.0      1
X-LC       XY25        95505.0      2
L-SC       OX40N       88966.0      1
H-SC       MH          85883.0      1
H-LC       AR          72257.0      1
M-MC       XY25        56375.0      1
L-MC       XR          54744.0      1
L-LC       XY25        39687.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
